# Data Engineer Project:  Automating Weekly Analytics Report
The objective was to create a script that reads in an Excel file (csv with all strings) that contains info from a fictional bank loan database.  Then, by typing in any start and end date, the script will give the user a range of useful analytics for that time period.  The dates on the file are between 01/01/2020 and 03/28/2020.  The script also factors in blank values and converts the data types when needed.    

Real world application:  Data analyst has to export data to Excel from a Customer Management System to constrcut a weekly report for senior management.  Instead of having to sort the data, apply multiple filters, and build a new report from scratch every week with new data, running this script will generate everything extremely quickly and accurately.  

Real world application two:  Analyst has to run various ad-hoc reports.  For instance, how many loans were closed during the first quarter, for the year, or what the current approved new loan pipeline is.  The script is flexible and allows the analyst to generate the information fast.     

For this project, only Python's built-in modules were used to show more in-depth code and how powerful the standard data structures can be.  

In [1]:
# importing the csv and datetime modules
import csv
from datetime import datetime

In [18]:
# reading in the Excel file.  Csv file with all the data coming in as a string data type.
# using a context manager for safer opening and closing.
with open('Test Loan Data File.csv', mode='r') as f: 
    rows = list(csv.reader(f))
    header = rows[0]
    file = rows[1:]

print(header) # printing the header row
print('\n')
for row in file[:5]:
    print(row,'\n') # printing a few rows of data.  The data columns and info are explained below this.
    

['Loan Type', 'Branch ID Number', 'Status', 'Closed Date', 'Renewal $ Amount', 'New $ Amount', 'Lender Name']


['New', '402', 'Funded', '1/1/2020', '', '375000', 'John'] 

['Renewal', '502', 'Approved', '', '450000', '', 'Nick'] 

['Renewal', '475', 'Funded', '3/28/2020', '350000', '', 'Jennifer'] 

['New', '402', 'Funded', '1/1/2020', '', '50000', 'John'] 

['New', '100', 'Funded', '1/1/2020', '', '50000', 'Nick'] 



# Explanation of the Data:

-Loan Type:  Whether the loan is a brand new deal (new) or renewing an existing loan (renewal)

-Branch ID Number:  The bank branch number that initiated the loan request

-Status:  Whether the loan closed (funded) or is approved but hasn't closed yet (approved)

-Closed Date:  Date the loan closed (only funded loans)

-Renewal $ Amount:  Dollar amount of the renewal loan (can be funded or approved status)

-New $ Amount:  Dollar amount of the new loan (can be funded or approved status)

-Lender Name:  Lender who worked on the request

-The empty ' ' means the data wouldn't be entered for that column.  Example:  if a loan was approved but didn't close, the closed date column would be empty.  

In [7]:
# creating a function that returns new loans that funded (closed) during a specific time frame.
def new_funded_dollars(start_date, end_date):
    total = 0
    count = 0
    for row in file:
        closed_date = row[3]
        if closed_date != '': # checking for blank values.
            begin_date = datetime.strptime(start_date, "%m/%d/%Y") # converting the string to a datetime object.
            last_date = datetime.strptime(end_date, "%m/%d/%Y") 
            date = datetime.strptime(row[3], "%m/%d/%Y")
        
        if (row[0] == 'New' 
        and row[2] == 'Funded'
        and row[5] != ''
        and date >= begin_date
        and date <= last_date):
            new_loan_amount = int(row[5]) # converting loan amount from a string to an integer.  
            total += new_loan_amount
            count += 1
    print(f"The total new funded dollars from {start_date} through {end_date} is: ${total:,d} and number of loans: {count}")

In [8]:
new_funded_dollars(start_date='01/01/2020', end_date='03/01/2020')

The total new funded dollars from 01/01/2020 through 03/01/2020 is: $3,830,000 and number of loans: 20


In [19]:
# creating a function that returns the number of loans a branch funded (closed) within the desired time frame.
# includes both new and renewal closed loans.  
# also orders the data by branch id number in ascending order.  The lowest branch id number will display first.
def loans_by_branch(start_date, end_date):
    new_dictionary = {}
    new_count = {}
    renewal_dictionary = {}
    renewal_count = {}
    
    for row in file:
        closed_date = row[3]
        if (closed_date != ''
        and row[0] == 'New'):
            # converts the start_date argument from a string into a datetime object.
            begin_date = datetime.strptime(start_date, "%m/%d/%Y")
            # converts the end_date argument into a datetime object.
            last_date = datetime.strptime(end_date, "%m/%d/%Y")
            # assigns the closed date column into a datetime object. 
            date = datetime.strptime(row[3], "%m/%d/%Y")
            branch_id = row[1]
        
        if date >= begin_date and date <= last_date and row[0] == 'New' and row[2] == 'Funded' and row[5] != '':
            new_loan_amount = int(row[5])
            if branch_id not in new_dictionary:
                new_dictionary[branch_id] = 1
                new_count[branch_id] = new_loan_amount
            else:
                new_dictionary[branch_id] += 1
                new_count[branch_id] += new_loan_amount
        
        elif date >= begin_date and date <= last_date and row[0] == 'Renewal' and row[2] == 'Funded' and row[4] != '':
            renewal_amount = int(row[4])
            if branch_id not in renewal_dictionary:
                renewal_dictionary[branch_id] = 1
                renewal_count[branch_id] = renewal_amount
            else:
                renewal_dictionary[branch_id] += 1
                renewal_count[branch_id] += renewal_amount
    
    print(f"For the time period of {start_date} through {end_date}:")
    
    # unpacking all items in one dictionary and just the values in the second dictionary.
    for (k, v), (v2) in sorted(zip(new_dictionary.items(), new_count.values())):  
        print(f"Branch ID number {k} closed {v} new loan/s for a total of ${v2:,d}")
    print('\n')
    
    for (k, v), (v2) in sorted(zip(renewal_dictionary.items(), renewal_count.values())):
        print(f"Branch ID number {k} closed {v} renewal loan/s for a total of ${v2:,d}")

In [20]:
loans_by_branch('01/01/2020', '03/20/2020')

For the time period of 01/01/2020 through 03/20/2020:
Branch ID number 100 closed 8 new loan/s for a total of $1,880,000
Branch ID number 200 closed 2 new loan/s for a total of $100,000
Branch ID number 402 closed 4 new loan/s for a total of $850,000
Branch ID number 405 closed 2 new loan/s for a total of $250,000
Branch ID number 475 closed 2 new loan/s for a total of $270,000
Branch ID number 538 closed 2 new loan/s for a total of $480,000


Branch ID number 100 closed 2 renewal loan/s for a total of $90,000
Branch ID number 200 closed 6 renewal loan/s for a total of $610,000
Branch ID number 402 closed 2 renewal loan/s for a total of $700,000
Branch ID number 538 closed 4 renewal loan/s for a total of $380,000


In [11]:
# creates a function that returns the number of current approved new and renewal loans.  
def active_pipeline():
    new_loan_total = 0
    new_loan_count = 0
    renewal_loan_total = 0
    renewal_loan_count = 0
    
    for row in file:
        status = row[2]
        if (status == 'Approved'
        and row[0] == 'New'
        and row[5] != ''):
            new_loan_amount = int(row[5])
            new_loan_total += new_loan_amount
            new_loan_count += 1
        
        if (status == 'Approved'
        and row[0] == 'Renewal'
        and row[4] != ''):
            renewal_loan_amount = int(row[4])
            renewal_loan_total += renewal_loan_amount
            renewal_loan_count += 1
    
    print(f"The current new approved loan pipeline is: ${new_loan_total:,d} and the number of loans is: {new_loan_count}")
    print(f"The current renewal approved loan pipeline is: ${renewal_loan_total:,d} and the number of loans is: {renewal_loan_count}")
    

In [12]:
active_pipeline()

The current new approved loan pipeline is: $2,230,000 and the number of loans is: 8
The current renewal approved loan pipeline is: $2,300,000 and the number of loans is: 6


In [14]:
# creates a function that returns the closed (funded) amount and number of loans for each lender in a given time period.
# filters for both new and renewal loans and the last line sums up the total during the time period.
# Sorts the data by total $ amount closed by lender in decending order.  The lender who closed the largest amount is first.
def loans_by_lender(start_date, end_date):
    new_loan_dictionary = {}
    renewal_dictionary = {}
    new_loan_count = {}
    renewal_loan_count = {}
    
    for row in file:
        closed_date = row[3]
        if closed_date != '':
            # converts the start_date parameter from a string into a datetime object.
            begin_date = datetime.strptime(start_date, '%m/%d/%Y')
            # converts the end_date into a datetime object
            last_date = datetime.strptime(end_date, '%m/%d/%Y')
            # assigns the closed date column into a datetime object.
            date = datetime.strptime(row[3], '%m/%d/%Y')
            # if statement to filter for only new loans during the start_date and end_date parameters
            lender_name = row[-1]
        
        if date >= begin_date and date <= last_date and row[2] == 'Funded' and row[5] != '' and row[0] == 'New':
            new_amount = int(row[5])
            if lender_name not in new_loan_dictionary:
                new_loan_dictionary[lender_name] = new_amount
                new_loan_count[lender_name] = 1
            else:
                new_loan_dictionary[lender_name] += new_amount
                new_loan_count[lender_name] += 1
        
        elif date >= begin_date and date <= last_date and row[2] == 'Funded' and row[4] != '' and row[0] == 'Renewal':
            renewal_amount = int(row[4])
            if lender_name not in renewal_dictionary:
                renewal_dictionary[lender_name] = renewal_amount
                renewal_loan_count[lender_name] = 1
            else:
                renewal_dictionary[lender_name] += renewal_amount
                renewal_loan_count[lender_name] += 1
    
    # sorts the data by dictionary values in desencing order (labmda function sorts by the dictionary values).            
    sorted_1 = sorted(new_loan_dictionary.items(), key=lambda x: x[1], reverse=True)
    # sorted function generates a list.  This converts the data back into a dictionary.  
    new_loan_dictionary = dict(sorted_1)
    
    sorted_2 = sorted(renewal_dictionary.items(), key=lambda x: x[1], reverse=True)
    renewal_dictionary = dict(sorted_2)
    
    print(f"For the time period {start_date} through {end_date}:")
    for (k, v), (v2) in zip(new_loan_dictionary.items(), new_loan_count.values()): # unpacking two dictionaries.
        print(f"{k} closed {v2} new loan/s totaling ${v:,d}")
    print('\n')    
    
    print(f"The total new funded amount from {start_date} through {end_date} is: ${sum(new_loan_dictionary.values()):,d} and number of loans is: {sum(new_loan_count.values())}")
    print('\n')
    
    print(f"For the time period {start_date} through {end_date}:")
    for (k, v), (k2, v2) in zip(renewal_dictionary.items(), renewal_loan_count.items()):
        print(f"{k} closed {v2} renewal loan/s totaling ${v:,d}")
    print('\n')    
    
    print(f"The total funded renewal amount from {start_date} through {end_date} is: ${sum(renewal_dictionary.values()):,d} and number of loans is: {sum(renewal_loan_count.values())}")
            
        

In [15]:
loans_by_lender('01/01/2020', '03/01/2020')

For the time period 01/01/2020 through 03/01/2020:
John closed 8 new loan/s totaling $2,000,000
Nick closed 4 new loan/s totaling $780,000
Sam closed 2 new loan/s totaling $680,000
Jennifer closed 4 new loan/s totaling $270,000
Steve closed 2 new loan/s totaling $100,000


The total new funded amount from 01/01/2020 through 03/01/2020 is: $3,830,000 and number of loans is: 20


For the time period 01/01/2020 through 03/01/2020:
Jennifer closed 4 renewal loan/s totaling $340,000
Nick closed 2 renewal loan/s totaling $130,000


The total funded renewal amount from 01/01/2020 through 03/01/2020 is: $470,000 and number of loans is: 6


When run all together, the following script would be simple to use for the data analyst and work as more data gets added to the database.  The user just needs to update the date and can run different time periods.  The analyst can also put in different dates within each individual function and it works as well.   

In [16]:
new_funded_dollars(start_date='01/01/2020', end_date='03/20/2020')
print('\n')
loans_by_branch(start_date='01/01/2020', end_date='03/20/2020')
print('\n')
active_pipeline()
print('\n')
loans_by_lender(start_date='01/01/2020', end_date='03/20/2020')

The total new funded dollars from 01/01/2020 through 03/20/2020 is: $3,830,000 and number of loans: 20


For the time period of 01/01/2020 through 03/20/2020:
Branch ID number 100 closed 8 new loan/s for a total of $1,880,000
Branch ID number 200 closed 2 new loan/s for a total of $100,000
Branch ID number 402 closed 4 new loan/s for a total of $850,000
Branch ID number 405 closed 2 new loan/s for a total of $250,000
Branch ID number 475 closed 2 new loan/s for a total of $270,000
Branch ID number 538 closed 2 new loan/s for a total of $480,000


Branch ID number 100 closed 2 renewal loan/s for a total of $90,000
Branch ID number 200 closed 6 renewal loan/s for a total of $610,000
Branch ID number 402 closed 2 renewal loan/s for a total of $700,000
Branch ID number 538 closed 4 renewal loan/s for a total of $380,000


The current new approved loan pipeline is: $2,230,000 and the number of loans is: 8
The current renewal approved loan pipeline is: $2,300,000 and the number of loans is: 

In [17]:
# different date with one function:
loans_by_lender(start_date='01/15/2020', end_date='02/25/2020')

For the time period 01/15/2020 through 02/25/2020:
Nick closed 2 new loan/s totaling $680,000
John closed 2 new loan/s totaling $250,000


The total new funded amount from 01/15/2020 through 02/25/2020 is: $930,000 and number of loans is: 4


For the time period 01/15/2020 through 02/25/2020:
Jennifer closed 4 renewal loan/s totaling $340,000
Nick closed 2 renewal loan/s totaling $130,000


The total funded renewal amount from 01/15/2020 through 02/25/2020 is: $470,000 and number of loans is: 6
